In [1]:
import requests
import pandas as pd
import time

from datetime import date
from datetime import timedelta

pd.set_option("display.max_columns", None)

In [2]:
from functions import make_request
from functions import get_recent_games
from functions import clean_stats
from functions import aggregate_stats

In [3]:
teams = make_request("teams", record_path="data")

team_codes = teams[["id", "name"]]

team_codes

,id,name
0,1,Hawks
1,2,Celtics
2,3,Nets
3,4,Hornets
4,5,Bulls
5,6,Cavaliers
6,7,Mavericks
7,8,Nuggets
8,9,Pistons
9,10,Warriors


In [4]:
# Get a pandas dataframe of the 20 most recent games for the following team
home_team_id = 1

today = date.today()                                                           # Get today
today = f"{today.year}-{today.month}-{today.day}"                              # Convert to format yyyy-mm-dd
one_year_ago = date.today() - timedelta(days=365)                              # Get last-year-today
one_year_ago = f"{one_year_ago.year}-{one_year_ago.month}-{one_year_ago.day}"  # convert to format yyyy-mm-dd

recent_games = pd.DataFrame()
res = make_request("games", record_path="data", params={"end_date": today,
                                                        "start_date": one_year_ago,
                                                        "team_ids[]": [home_team_id],
                                                        "page": "1",
                                                       "per_page": "100"})

res = res.sort_values("date", ascending=False)
res = res[res["home_team.id"].eq(home_team_id)]

recent_games = recent_games.append(res)
                                ### Should write a line of code here that drops the row if "time" is not an empty string (meaning the game is still in progress)
recent_games = recent_games.head(20)

In [5]:
recent_games

,id,date,home_team_score,period,postseason,season,status,time,visitor_team_score,home_team.id,home_team.abbreviation,home_team.city,home_team.conference,home_team.division,home_team.full_name,home_team.name,visitor_team.id,visitor_team.abbreviation,visitor_team.city,visitor_team.conference,visitor_team.division,visitor_team.full_name,visitor_team.name
46,473663,2021-11-22T00:00:00.000Z,113,4,False,2021,Final,,101,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,21,OKC,Oklahoma City,West,Northwest,Oklahoma City Thunder,Thunder
37,473652,2021-11-20T00:00:00.000Z,115,4,False,2021,Final,,105,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,4,CHA,Charlotte,East,Southeast,Charlotte Hornets,Hornets
56,473621,2021-11-17T00:00:00.000Z,110,4,False,2021,Final,,99,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
43,473607,2021-11-15T00:00:00.000Z,129,4,False,2021,Final,,111,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,22,ORL,Orlando,East,Southeast,Orlando Magic,Magic
55,473603,2021-11-14T00:00:00.000Z,120,4,False,2021,Final,,100,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,17,MIL,Milwaukee,East,Central,Milwaukee Bucks,Bucks
52,473528,2021-11-04T00:00:00.000Z,98,4,False,2021,Final,,116,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,29,UTA,Utah,West,Northwest,Utah Jazz,Jazz
47,473505,2021-11-01T00:00:00.000Z,118,4,False,2021,Final,,111,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,30,WAS,Washington,East,Southeast,Washington Wizards,Wizards
44,473454,2021-10-25T00:00:00.000Z,122,4,False,2021,Final,,104,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,9,DET,Detroit,East,Central,Detroit Pistons,Pistons
35,473424,2021-10-21T00:00:00.000Z,113,4,False,2021,Final,,87,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,7,DAL,Dallas,West,Southwest,Dallas Mavericks,Mavericks
2,459132,2021-08-12T00:00:00.000Z,88,5,False,2021,Final,,96,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,23,PHI,Philadelphia,East,Atlantic,Philadelphia 76ers,76ers


In [6]:
game_ids_home, game_ids_away = get_recent_games(1,2)

In [7]:
game_ids_home

[473663,
 473652,
 473621,
 473607,
 473603,
 473528,
 473505,
 473454,
 473424,
 459132,
 451506,
 446517,
 445510,
 444934,
 442056,
 440182,
 438743,
 430992,
 429183,
 264794]

In [8]:
def get_stats(game_ids_home, game_ids_away):
    
    def format_params(game_ids):
        """ 
        Format query paramaters in a format the balldontlie API accepts
        e.g. ?game_ids[]=345686&game_ids[]=234356&gameids[]=3456356...
        """
        params = "game_ids[] " * len(game_ids_home)
        params = list(zip(params.split(" "), game_ids_home))
        params.append(("per_page", 100))
        return params
    
    # Get pandas dataframe of home team stats
    params_home = format_params(game_ids_home)                                 # Get param list
    stats_home = make_request("stats", record_path="data", params=params_home) # Make request with said param list
    stats_home = clean_stats(stats_home)                                       # clean the data
    stats_home = stats_home[stats_home["team.id"].eq(stats_home["game.home_team_id"])]  # Filter for stats of players that played for the home team
    stats_home = aggregate_stats(stats_home)                                   # aggregate individual player stats into team stats
    
    # Get pandas dataframe of away team stats
    params_away = format_params(game_ids_away)
    stats_away = make_request("stats", record_path="data", params=params_away)
    stats_away = clean_stats(stats_away)
    stats_away = stats_away[stats_away["team.id"].eq(stats_away["game.home_team_id"])]
    stats_away = aggregate_stats(stats_away)
    
    return stats_home, stats_away

In [9]:
stats_home, stats_away = get_stats(game_ids_home, game_ids_away)

/Users/zacharyargentin/Programming/Bootcamp/final/basketball/functions.py:110: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["min"] = df["min"].str.replace(".",":")
/Users/zacharyargentin/opt/anaconda3/envs/bootcamp/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
stats_home.head(10)

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,ft_pct,fta,ftm,min,oreb,pf,pts,reb,stl,turnover,game.id,game.date,game.home_team_id,game.home_team_score,game.period,game.postseason,game.season,game.status,game.time,game.visitor_team_id,game.visitor_team_score,player.id,player.first_name,player.height_feet,player.height_inches,player.last_name,player.position,player.team_id,player.weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name
0,6528101,6,0,2,0.0,0,0,66.7,6,4,100.0,3,3,16:33,0,1,11,2,0,4,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,481,Lou,6.0,1.0,Williams,G,1,175.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
1,6528082,5,0,3,40.0,10,4,45.5,11,5,100.0,4,4,22:01,1,1,18,4,1,2,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,21,D.J.,6.0,0.0,Augustin,G,11,183.0,11,HOU,Houston,West,Southwest,Houston Rockets,Rockets
2,6528080,4,1,5,40.0,5,2,27.3,11,3,100.0,8,8,23:59,1,3,16,6,1,2,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,358,Kelly,7.0,0.0,Olynyk,F,9,240.0,11,HOU,Houston,West,Southwest,Houston Rockets,Rockets
3,6528100,1,0,2,0.0,0,0,57.1,7,4,0.0,0,0,23:33,0,3,8,2,2,2,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,426,Tony,6.0,7.0,Snell,G,25,213.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
4,6528093,4,0,4,33.3,6,2,28.6,7,2,0.0,0,0,35:47,1,2,6,5,2,1,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,212,Solomon,6.0,7.0,Hill,F,1,225.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
5,6528089,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,,0,0,0,0,0,0,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,486,Christian,6.0,10.0,Wood,F,11,214.0,11,HOU,Houston,West,Southwest,Houston Rockets,Rockets
6,6528095,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,,0,0,0,0,0,0,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,139,Kris,6.0,4.0,Dunn,G,15,205.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
7,6528084,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,,0,0,0,0,0,0,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,2175,Danuel,6.0,7.0,House Jr.,,11,220.0,11,HOU,Houston,West,Southwest,Houston Rockets,Rockets
8,6528091,0,1,4,100.0,1,1,71.4,7,5,100.0,4,4,16:20,0,1,15,4,0,0,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,101,John,6.0,10.0,Collins,F-C,1,235.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
9,6528088,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0:00,0,0,0,0,0,0,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,484,D.J.,6.0,10.0,Wilson,F,21,231.0,11,HOU,Houston,West,Southwest,Houston Rockets,Rockets


In [11]:
stats_home = clean_stats(stats_home)                                       # clean the data
stats_home = stats_home[stats_home["team.id"].eq(stats_home["game.home_team_id"])]  # Filter for stats of players that played for the home team
stats_home = aggregate_stats(stats_home)    

/Users/zacharyargentin/Programming/Bootcamp/final/basketball/functions.py:110: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["min"] = df["min"].str.replace(".",":")
/Users/zacharyargentin/opt/anaconda3/envs/bootcamp/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [15]:
stats_home

,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,ft_pct,fta,ftm,min,oreb,pf,pts,reb,stl,turnover,game.id,game.date,game.season,game.home_team_id,game.home_team_score,game.visitor_team_id,game.visitor_team_score,player.id,team.id
game.id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
264794,17,3,22,33.320000,10,4,53.460000,43,24,41.660000,11,8,0 days 02:04:54,7,8,60,29,8,6,264794,2021-05-16,2020,1,124,11,95,426,1
429183,6,0,12,56.250000,11,8,41.675000,27,14,0.000000,0,0,0 days 01:18:38,1,9,36,13,2,5,429183,2021-05-28,2020,1,105,20,94,481,1
430992,11,0,7,31.433333,21,8,25.000000,35,14,33.333333,7,7,0 days 01:21:53,3,12,43,10,1,4,430992,2021-05-30,2020,1,113,20,96,212,1
438743,1,0,7,6.666667,11,1,44.166667,29,15,66.666667,12,12,0 days 01:25:56,3,11,43,10,0,3,438743,2021-06-11,2020,1,111,23,127,167,1
440182,3,2,22,16.025000,19,5,36.050000,51,21,12.500000,3,1,0 days 02:01:54,8,7,48,30,1,2,440182,2021-06-14,2020,1,103,23,100,426,1
442056,12,0,5,50.000000,10,5,43.300000,30,13,60.000000,5,3,0 days 00:40:10,0,4,34,5,3,4,442056,2021-06-18,2020,1,99,23,104,490,1
444934,5,2,12,21.675000,13,4,31.475000,37,14,25.000000,6,2,0 days 01:53:56,7,9,34,19,6,5,444934,2021-06-27,2020,1,102,17,113,167,1
445510,16,2,22,26.200000,30,12,35.337500,56,25,22.912500,8,7,0 days 02:26:56,3,15,69,25,8,8,445510,2021-06-29,2020,1,110,17,88,481,1
446517,7,0,4,51.200000,13,7,39.150000,22,9,25.000000,2,1,0 days 00:58:09,2,4,26,6,5,2,446517,2021-07-03,2020,1,107,17,118,221,1


In [16]:
stats_home["team.id"].value_counts()

1    20
Name: team.id, dtype: int64

In [14]:
stats_away

,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,ft_pct,fta,ftm,min,oreb,pf,pts,reb,stl,turnover,game.id,game.date,game.home_team_id,game.home_team_score,game.season,game.visitor_team_id,game.visitor_team_score,player.id,team.id
1,5,0,3,40.0,10,4,45.5,11,5,100.0,4,4,22:01,1,1,18,4,1,2,264794,2021-05-16T00:00:00.000Z,1,124,2020,11,95,21,11
3,1,0,2,0.0,0,0,57.1,7,4,0.0,0,0,23:33,0,3,8,2,2,2,264794,2021-05-16T00:00:00.000Z,1,124,2020,11,95,426,1
4,4,0,4,33.3,6,2,28.6,7,2,0.0,0,0,35:47,1,2,6,5,2,1,264794,2021-05-16T00:00:00.000Z,1,124,2020,11,95,212,1
8,0,1,4,100.0,1,1,71.4,7,5,100.0,4,4,16:20,0,1,15,4,0,0,264794,2021-05-16T00:00:00.000Z,1,124,2020,11,95,101,1
13,9,0,3,33.3,3,1,33.3,9,3,75.0,4,3,16:20,0,0,10,3,2,2,264794,2021-05-16T00:00:00.000Z,1,124,2020,11,95,490,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,3,0,0,50.0,2,1,50.0,6,3,50.0,2,1,16:57,0,1,8,0,0,2,473663,2021-11-22T00:00:00.000Z,1,113,2021,21,101,481,1
47,5,5,5,33.3,3,1,50.0,8,4,100.0,2,2,34:54,1,2,11,6,1,1,473663,2021-11-22T00:00:00.000Z,1,113,2021,21,101,101,1
51,1,0,2,25.0,4,1,36.4,11,4,0.0,0,0,22:59,1,1,9,3,1,0,473663,2021-11-22T00:00:00.000Z,1,113,2021,21,101,666860,1
55,0,0,2,0.0,0,0,50.0,4,2,0.0,0,0,10:20,1,0,4,3,0,1,473663,2021-11-22T00:00:00.000Z,1,113,2021,21,101,666873,21
